Quantium Virtual Internship - Retail Strategy and Analytics - Task 2

In [2]:
import pandas as pd
data = pd.read_csv('data/QVI_data.csv')

In [3]:
data.head(4)

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,PACK_SIZE,BRANDS,LIFESTAGE,PREMIUM_CUSTOMER
0,2018-10-17,1,1000,1,5,Natural Chip Compny SeaSalt175g,2,6.0,175,NCC,YOUNG SINGLES/COUPLES,Premium
1,2019-05-14,1,1307,348,66,CCs Nacho Cheese 175g,3,6.3,175,CCs,MIDAGE SINGLES/COUPLES,Budget
2,2019-05-20,1,1343,383,61,Smiths Crinkle Cut Chips Chicken 170g,2,2.9,170,Smith,MIDAGE SINGLES/COUPLES,Budget
3,2018-08-17,2,2373,974,69,Smiths Chip Thinly S/Cream&Onion 175g,5,15.0,175,Smith,MIDAGE SINGLES/COUPLES,Budget


The client wants to evaluate the performance of a store trial which was performed in stores 77, 86 and 88.
 and want control stores to be established stores that are operational for the entire observation period. We would want to match trial stores to control stores that are similar to the trial
store prior to the trial period of Feb 2019 in terms of :
- Monthly overall sales revenue
- Monthly number of customers
- Monthly number of transactions per customer

First create the metrics of interest and filter to stores that are present
throughout the pre-trial period.


In [5]:
# Convert the 'DATE' column to a datetime format to extract month and year
data['DATE'] = pd.to_datetime(data['DATE'])

In [8]:
#extract month and year
data['month_id'] = data['DATE'].dt.strftime('%Y%m')

In [17]:
#calculate number of unique months for each store
store_months_counts = data.groupby('STORE_NBR')['month_id'].nunique()
store_months_counts.head(3)

STORE_NBR
1    12
2    12
3    12
Name: month_id, dtype: int64

In [20]:
#here the stores with 12 months show there where present before Feb 2019
# thus should be filtered out to be used for pre trial period. the data is a list of the store numbers
stores_present_pre_trial = store_months_counts[store_months_counts == 12].index

In [21]:
# Filter the DataFrame to include only the selected stores
data_pre_trial = data[data['STORE_NBR'].isin(stores_present_pre_trial)]

In [22]:
data_pre_trial.head(2)

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,PACK_SIZE,BRANDS,LIFESTAGE,PREMIUM_CUSTOMER,month_id
0,2018-10-17,1,1000,1,5,Natural Chip Compny SeaSalt175g,2,6.0,175,NCC,YOUNG SINGLES/COUPLES,Premium,201810
1,2019-05-14,1,1307,348,66,CCs Nacho Cheese 175g,3,6.3,175,CCs,MIDAGE SINGLES/COUPLES,Budget,201905


In [27]:
# Calculate total sales, number of customers, transactions per customer, chips per transaction, and average price per unit
# convert DATE to datetime to ensure consistent date

measureOverTime = data.groupby(['STORE_NBR', 'DATE']).agg(
    totSales=('TOT_SALES', 'sum'),
    noCustomers=('LYLTY_CARD_NBR', 'nunique'),
    noTxnPerCust=('TXN_ID', 'nunique'),
    noChipsPerTxn=('PROD_QTY', 'sum'),
).reset_index()

#calculate average price per unit
measureOverTime['avgPriceUnit'] = measureOverTime['totSales'] / measureOverTime['noChipsPerTxn']

measureOverTime.head(4)

,STORE_NBR,DATE,totSales,noCustomers,noTxnPerCust,noChipsPerTxn,avgPriceUnit
0,1,2018-07-01,4.8,2,2,2,2.400000
1,1,2018-07-02,10.6,3,3,3,3.533333
2,1,2018-07-03,1.9,1,1,1,1.900000
3,1,2018-07-04,10.8,1,1,2,5.400000


In [31]:
# Filter to the pre-trial period and stores with full observation periods
storesWithFullObs = measureOverTime[measureOverTime.groupby('STORE_NBR')['DATE'].transform('nunique') == 12]['STORE_NBR'].unique()
preTrialMeasures = measureOverTime[(measureOverTime['DATE'] < '2019-02-01') & ( measureOverTime['STORE_NBR'].isin(storesWithFullObs))]

In [34]:
#  to work out a function that ranks the correlation each potential control store
# is to the trial store. We can calculate how correlated the performance of each
# control store is to the trial store.

def calculate_correlation(input_table, metric_col, store_comparison):
    #setting the empty data frame
    calc_corr_table = pd.DataFrame(columns=['Store1', 'Store2', 'corr_measure'])
    #get store numbers 
    store_numbers = input_table['STORE_NBR'].unique()
    
    for control_store in store_numbers:
        if control_store != trial_store:
            #use loc label based indexing to findout for store numbers either in
            #trial_store or control store. this will pivot if true and proceed to pivot but will
            #give false if store_nbr is found in both stores
            correlation = input_table.loc[
                input_table['STORE_NBR'].isin([trial_store, control_store])
            ].pivot(
                index='DATE', columns='STORE_NBR', values=metric_col
            ).corr().iloc[0, 1]
            
            calc_corr_table = cal_corr_table.append(
                {'Store1': trial_store, 'Store2': control_store, 'corr_measure': correlation},
                ignore_index=True
            )
            
        
    return cal_corr_table



In [35]:
#  to work out a function that ranks the magnitude distance each potential control store
# is to the trial store.

def calculate_magnitude_distance(input_table, metric_col, trial_store):
    calc_dist_table = pd.DataFrame(columns=['Store1', 'Store2', 'YEARMONTH', 'measure'])
    store_numbers = input_table['STORE_NBR'].unique()
    
    for control_store in store_numbers:
        if control_store != trial_store:
            calculated_measure = pd.DataFrame({
                'Store1': trial_store,
                'Store2': control_store,
                'YEARMONTH': input_table.loc[input_table['STORE_NBR'] == trial_store, 'YEARMONTH'],
                'measure': abs(input_table.loc[input_table['STORE_NBR'] == trial_store, metric_col].values - 
                               input_table.loc[input_table['STORE_NBR'] == control_store, metric_col].values)
            })
            
            calc_dist_table = cal_dist_table.append(calculated_measure, ignore_index=True)
            

    # Standardize the magnitude distance
    min_max_dist = calc_dist_table.groupby(['Store1', 'YEARMONTH'])['measure'].agg(['min', 'max']).reset_index()
    dist_table = pd.merge(calc_dist_table, min_max_dist, on=['Store1', 'YEARMONTH'])
    dist_table['magnitudeMeasure'] = 1 - (dist_table['measure'] - dist_table['min']) / (dist_table['max'] - dist_table['min'])
    
    final_dist_table = dist_table.groupby(['Store1', 'Store2'])['magnitudeMeasure'].mean().reset_index()
    return final_dist_table